<a href="https://colab.research.google.com/github/yashkens/ML_Intro_homework/blob/main/HW2_Regression_Texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

На этом занятии мы попробуем задачу регрессии. Данные в этой же папке, будем тренироваться на датасете фильмов с IMDB

Перед обучением обучением модели, нужно подготовить данные:

- найти\собрать данные
- почистить и предобработать
- преобразовать в матрицы 


In [ ]:
# импорты необходимых библиотек
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [ ]:
data = pd.read_csv('IMDB-Movie-Data.csv')
print(data.shape)

data.head(3)

(1000, 12)


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0


## Что делать с NaN?
Есть 3 варианта

In [ ]:
# 1. Убрать строки с NaN
print(data.isna().any())
data.shape

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)     True
Metascore              True
dtype: bool


(1000, 12)

In [ ]:
print(data.shape)
tmp = data.dropna()
tmp.shape

(1000, 12)


(838, 12)

In [ ]:
# 2. Превратить NaN в 0
print(data.shape)
tmp = data.fillna(0)
print(tmp.shape)

(1000, 12)
(1000, 12)


In [ ]:
# 3. Превратить NaN в средние значения по колонке

# вычисляем средние для колонок с пустыми значениями
meta_mean = data.Metascore.mean()
rev_mean = data['Revenue (Millions)'].mean()

#заменяем пустоты на средние значения
data.Metascore.fillna(meta_mean, inplace=True)
data['Revenue (Millions)'].fillna(rev_mean, inplace=True)

# проверяем присутствие NaN
data.isna().any()

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
dtype: bool

## Подготовка данных

Попробуем предсказывать рейтинг фильма по данным его описания, года, длины в минутах и кассовых сборов

Колонка "Rating" станет **целевой переменной, или таргетом** (y)<br>
Остальных данные будут **обучающей выборкой** (X)

In [ ]:
data.Description

0      A group of intergalactic criminals are forced ...
1      Following clues to the origin of mankind, a te...
2      Three girls are kidnapped by a man with a diag...
3      In a city of humanoid animals, a hustling thea...
4      A secret government agency recruits some of th...
                             ...                        
995    A tight-knit team of rising investigators, alo...
996    Three American college students studying abroa...
997    Romantic sparks occur between two dance studen...
998    A pair of friends embark on a mission to reuni...
999    A stuffy businessman finds himself trapped ins...
Name: Description, Length: 1000, dtype: object

In [ ]:
# подготовим описания фильмов
data["text"] = data.Description.apply(lambda x: x.lower().split()) 

data["text"]

0      [a, group, of, intergalactic, criminals, are, ...
1      [following, clues, to, the, origin, of, mankin...
2      [three, girls, are, kidnapped, by, a, man, wit...
3      [in, a, city, of, humanoid, animals,, a, hustl...
4      [a, secret, government, agency, recruits, some...
                             ...                        
995    [a, tight-knit, team, of, rising, investigator...
996    [three, american, college, students, studying,...
997    [romantic, sparks, occur, between, two, dance,...
998    [a, pair, of, friends, embark, on, a, mission,...
999    [a, stuffy, businessman, finds, himself, trapp...
Name: text, Length: 1000, dtype: object

In [ ]:
# data.text.values

In [ ]:
input_text = list(data.text.values)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
documents[10:12]

[TaggedDocument(words=['the', 'adventures', 'of', 'writer', 'newt', 'scamander', 'in', 'new', "york's", 'secret', 'community', 'of', 'witches', 'and', 'wizards', 'seventy', 'years', 'before', 'harry', 'potter', 'reads', 'his', 'book', 'in', 'school.'], tags=[10]),
 TaggedDocument(words=['the', 'story', 'of', 'a', 'team', 'of', 'female', 'african-american', 'mathematicians', 'who', 'served', 'a', 'vital', 'role', 'in', 'nasa', 'during', 'the', 'early', 'years', 'of', 'the', 'u.s.', 'space', 'program.'], tags=[11])]

обучаем модель на текстах описаний фильмов (можно поизменять параметры)

In [ ]:
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [ ]:
model.save("D2V.model") # сохранение модели

In [ ]:
# так можно посмотреть на векторы текстов, на которых училась модель
# индекс [] около documents -- это индекс текста из датасета

model[documents[0].tags[0]]

array([-0.01824311,  0.04826299, -0.01255307, -0.00534828,  0.08149271],
      dtype=float32)

Теперь нужно добавить векторы в датасет с остальными параметрами

In [ ]:
# создадим список с векторами для каждого текста
vectors = []
for x in documents:
    vec = list(model[x.tags][0])
    vectors.append(vec)

In [ ]:
# так получим датафрейм, где все компоненты векторов в отдельных столбцах
split_df = pd.DataFrame(vectors,
                        columns=['v1', 'v2', 'v3','v4',"v5"])

split_df.head()


,v1,v2,v3,v4,v5
0,-0.001076,0.042594,-0.057175,-0.029541,0.118134
1,0.080965,0.054143,-0.114492,-0.025028,0.066237
2,-0.045010,0.050534,-0.161250,-0.095246,0.068004
3,-0.010029,-0.004318,-0.165446,-0.067065,0.094697
4,0.044040,0.080643,-0.152245,0.049812,0.001349


In [ ]:
# теперь добавим его к основному датафрейму
result = data.join(split_df, how='left')
result.shape

(1000, 18)

In [ ]:
# result

In [ ]:
# переопределим датасет, оставив только важное

data_sm = result[['Runtime (Minutes)',"Year",
                'Rating', 'Votes',
                'Revenue (Millions)','Metascore',"v1","v2","v3","v4","v5"]
              ]


data_sm.head(3)

,Runtime (Minutes),Year,Rating,Votes,Revenue (Millions),Metascore,v1,v2,v3,v4,v5
0,121,2014,8.1,757074,333.13,76.0,-0.001076,0.042594,-0.057175,-0.029541,0.118134
1,124,2012,7.0,485820,126.46,65.0,0.080965,0.054143,-0.114492,-0.025028,0.066237
2,117,2016,7.3,157606,138.12,62.0,-0.045010,0.050534,-0.161250,-0.095246,0.068004


## Подготавливаем матрицы

In [ ]:
# определяем X и y

X = data_sm.drop(["Rating"],axis=1).values 

display(X, X.shape)

array([[ 1.21000000e+02,  2.01400000e+03,  7.57074000e+05, ...,
        -5.71754053e-02, -2.95406990e-02,  1.18134394e-01],
       [ 1.24000000e+02,  2.01200000e+03,  4.85820000e+05, ...,
        -1.14491880e-01, -2.50277519e-02,  6.62365109e-02],
       [ 1.17000000e+02,  2.01600000e+03,  1.57606000e+05, ...,
        -1.61250159e-01, -9.52458307e-02,  6.80036247e-02],
       ...,
       [ 9.80000000e+01,  2.00800000e+03,  7.06990000e+04, ...,
        -1.26663327e-01, -2.47532837e-02, -7.25467503e-03],
       [ 9.30000000e+01,  2.01400000e+03,  4.88100000e+03, ...,
        -4.19106409e-02,  5.17834798e-02,  1.09310322e-01],
       [ 8.70000000e+01,  2.01600000e+03,  1.24350000e+04, ...,
        -7.12807626e-02, -1.23197049e-01,  3.72172818e-02]])

(1000, 10)

In [ ]:
data_sm.isna().any()

Runtime (Minutes)     False
Year                  False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
v1                    False
v2                    False
v3                    False
v4                    False
v5                    False
dtype: bool

In [ ]:
y = data_sm['Rating'].values # отдельно вынесли массив со значениями скорости ветра
y.shape

(1000,)

Иногда бывает полезно [нормализовать](https://en.wikipedia.org/wiki/Normalization_(statistics)) данные: это позволяет исправить ситуацию, когда признаки представлены в разных единацах измерения. 
Для этого используется StandardScaler. 

До нормализации:

In [ ]:
list(X[0])

[121.0,
 2014.0,
 757074.0,
 333.13,
 76.0,
 -0.0010758353164419532,
 0.04259422421455383,
 -0.0571754053235054,
 -0.02954069897532463,
 0.11813439428806305]

In [ ]:
# использзуем стандартизатор
sc = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=42)

После:

In [ ]:
list(sc.fit_transform(X)[0])

[0.4163497512303056,
 0.37979525138136244,
 3.1126899627963738,
 2.5961363010556906,
 1.0233613578368184,
 -0.7647593611893714,
 0.6966789536735115,
 0.7662552330208103,
 0.18509959759690034,
 0.5528462220970306]

теперь с данными удобнее работать и обучать

In [ ]:
# задаем модель регрессора
# силу регуляризации можно варьировать параметром alpha
regressor = Ridge() 


# обучаем
regressor.fit(X_train, y_train)

Ridge()

In [ ]:
# давайте предскажем результат для тестовой выборки

y_preds = regressor.predict(X_test)

### оценка результатов алгоритма

В качестве метрики будем использовать [среднюю абсолютную ошибку](https://www.youtube.com/watch?v=ZejnwbcU8nw). Она показывает отклонение от правильного ответа в тех же единах измерения

*(а вообще есть [разные способы](https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b))*

In [ ]:
mean_absolute_error(y_test, y_preds) 

0.48985434083523205

Попробуйте разные значения для параметра регуляризации alpha при обучении модели. Как они влияют на величину ошибки?

##Домашка

### Ridge
попробуем разные alpha

In [ ]:
alphas = [0.0001, 0.1, 1, 10, 50, 100, 200]
best_alpha = 1
best_score = 10000

for alpha in alphas:
  regressor = Ridge(alpha=alpha) 
  regressor.fit(X_train, y_train)
  y_preds = regressor.predict(X_test)
  mae = mean_absolute_error(y_test, y_preds) 

  if mae < best_score:
    best_score = mae
    best_alpha = alpha

  print(f'Rigde with alpha={alpha}')
  print(f'MAE: {mae:.4f}')
  print('----------------')

print('----------------')
print(f'Best alpha = {best_alpha} with MAE = {best_score:.4f}')

Rigde with alpha=0.0001
MAE: 0.4898
----------------
Rigde with alpha=0.1
MAE: 0.4898
----------------
Rigde with alpha=1
MAE: 0.4899
----------------
Rigde with alpha=10
MAE: 0.4899
----------------
Rigde with alpha=50
MAE: 0.4912
----------------
Rigde with alpha=100
MAE: 0.4932
----------------
Rigde with alpha=200
MAE: 0.4977
----------------
----------------
Best alpha = 0.0001 with MAE = 0.4898


In [ ]:
regressor = Ridge(alpha=0.1) 
regressor.fit(X_train, y_train)
regressor.coef_

array([ 0.18200918, -0.02938208,  0.34780387, -0.11822411,  0.42964527,
       -0.00574946, -0.0246499 ,  0.00240736,  0.01078934,  0.00426747])

Чем больше альфа, тем хуже результат.  
Видимо, эти данные не страдают от тех проблем, с которыми борется Ridge регуляризация. Также видно, что веса не склонны быть слишком большими или маленькими, поэтому сильная регуляризация не нужна.

### Lasso
попробую разные alpha

In [ ]:
alphas = [1e-6, 0.0001, 0.1, 1, 10, 50, 100, 200]
best_alpha = 1
best_score = 10000

for alpha in alphas:
  regressor = Lasso(alpha=alpha) 
  regressor.fit(X_train, y_train)
  y_preds = regressor.predict(X_test)
  mae = mean_absolute_error(y_test, y_preds) 

  if mae < best_score:
    best_score = mae
    best_alpha = alpha

  print(f'Lasso with alpha={alpha}')
  print(f'MAE: {mae:.4f}')
  print('----------------')

print('----------------')
print(f'Best alpha = {best_alpha} with MAE = {best_score:.4f}')

Lasso with alpha=1e-06
MAE: 0.4898
----------------
Lasso with alpha=0.0001
MAE: 0.4898
----------------
Lasso with alpha=0.1
MAE: 0.4971
----------------
Lasso with alpha=1
MAE: 0.7736
----------------
Lasso with alpha=10
MAE: 0.7736
----------------
Lasso with alpha=50
MAE: 0.7736
----------------
Lasso with alpha=100
MAE: 0.7736
----------------
Lasso with alpha=200
MAE: 0.7736
----------------
----------------
Best alpha = 0.0001 with MAE = 0.4898


In [ ]:
regressor = Lasso(alpha=0.0001) 
regressor.fit(X_train, y_train)
regressor.coef_

array([ 0.18193523, -0.02930459,  0.34769581, -0.11806043,  0.42962044,
       -0.00562167, -0.02456635,  0.00228988,  0.0106889 ,  0.00406454])

In [ ]:
regressor = Lasso(alpha=100) 
regressor.fit(X_train, y_train)
regressor.coef_

array([ 0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0.])

Чем меньше alpha, тем лучше.  
Lasso действует жестче, зануляя веса незначимых признаков. Посмотрев на веса с маленькой и большой альфа, опять видим, что регуляризация нам скорее не нужна в этом случае, лучший результат достигнут вообще без зануления.

### Basic Linear Regression
посмотрим, справится ли без регуляризации

In [ ]:
regressor = LinearRegression() 
regressor.fit(X_train, y_train)
y_preds = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_preds) 

print(f'Linear Regression')
print(f'MAE: {mae:.4f}')

Linear Regression
MAE: 0.4898


In [ ]:
regressor.coef_

array([ 0.18201176, -0.0293538 ,  0.34789515, -0.11829647,  0.42968697,
       -0.00575309, -0.02464964,  0.00241142,  0.01079067,  0.00427256])

В общем, тут обойдемся без регуляризации

### Preprocessing

Попробую превращать тесты в вектора разного размера

In [ ]:
def prepare_features(input_text, vector_size=5):
  documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
  model = Doc2Vec(documents, vector_size=vector_size, window=2, min_count=1, workers=4)
  vectors = []
  for x in documents:
      vec = list(model[x.tags][0])
      vectors.append(vec)

  vec_columns = ["v" + str(i) for i in range(vector_size)]
  split_df = pd.DataFrame(vectors,
                        columns=vec_columns)
  result = data.join(split_df, how='left')
  
  features = ['Runtime (Minutes)', "Year", 'Rating', 
              'Votes','Revenue (Millions)','Metascore']
  features.extend(vec_columns)
  data_sm = result[features]
  return data_sm

In [ ]:
def preprocess_data(input_text, vector_size=5):
  data_sm = prepare_features(input_text, vector_size=vector_size)
  X = data_sm.drop(["Rating"], axis=1).values 
  y = data_sm['Rating'].values
  sc = StandardScaler()
  X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=42)
  return X_train, X_test, y_train, y_test

In [ ]:
sizes = [3, 5, 8, 12]
best_size = 3
best_score = 10000

for vec_size in sizes:
  X_train, X_test, y_train, y_test = preprocess_data(input_text, vector_size=vec_size)

  regressor = LinearRegression() 
  regressor.fit(X_train, y_train)
  y_preds = regressor.predict(X_test)
  mae = mean_absolute_error(y_test, y_preds) 

  if mae < best_score:
    best_score = mae
    best_size = vec_size

  print(f'Text vector size = {vec_size}')
  print(f'MAE: {mae:.4f}')
  print('----------------')

Text vector size = 3
MAE: 0.4899
----------------


Text vector size = 5
MAE: 0.4900
----------------


Text vector size = 8
MAE: 0.4899
----------------


Text vector size = 12
MAE: 0.4869
----------------


In [ ]:
print(f'Best vector size = {best_size} with MAE: {best_score:.4f}')

Best vector size = 12 with MAE: 0.4869


На этой маленькой выборке размеров результаты очень сравнимы, но с небольшим отрывом выиграл размер равный 12.

Еще можно убрать стоп слова

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_stopwords(text):
  good_tokens = []
  for token in text:
    if token not in stop_words:
      good_tokens.append(token)
  return good_tokens

In [ ]:
remove_stopwords(data["text"][0])

['group',
 'intergalactic',
 'criminals',
 'forced',
 'work',
 'together',
 'stop',
 'fanatical',
 'warrior',
 'taking',
 'control',
 'universe.']

In [ ]:
data["text"] = data.Description.apply(lambda x: x.lower().split()) 
input_text = list(data.text.values)

final_texts = []
for text in input_text:
  final_texts.append(remove_stopwords(text))

In [ ]:
X_train, X_test, y_train, y_test = preprocess_data(input_text, vector_size=5)

regressor = LinearRegression() 
regressor.fit(X_train, y_train)
y_preds = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_preds) 

print('Linear Regression with removed stop words')
print(f'MAE: {mae:.4f}')

Linear Regression with removed stop words
MAE: 0.4895


Не дало значительного улучшения, но я пыталась!